# Claim Approval Process


In [1]:
# To ignore any warnings
import warnings
warnings.filterwarnings('ignore')
#To run all the commands in a same shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib import pyplot
import numpy as np
import pandas as pd
from scipy.stats import norm
# For plotting graphs
import matplotlib.pyplot as plt

%matplotlib inline
# For data visualization
import seaborn as sns
#for different algorithms
from sklearn.utils import shuffle
from factor_analyzer import FactorAnalyzer
from sklearn import decomposition, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Import Dataset

In [2]:
df= pd.read_csv((r'C:/Users/Pavan/Documents/Insurance/TrainingData_New.csv'))


In [3]:
df.Claimstatus=df.Claimstatus.map({'C':1,'D':0}) 
df.isnull().sum()

ClaimID                 0
AGMID                   0
LOB                     0
State             1604742
ProgramCode             0
PlanCode                0
VehicleMake           123
VehicleModel         3829
VehicleYear             0
TermMonth               0
TermMiles               0
ComponentCode           0
ComponentSet           82
ContractStatus          0
AgeAtClaim              2
AgeAtCancel             0
Claimstatus       2505572
ClaimDate             382
PaidDate                0
AgeofPayment            0
dtype: int64

In [4]:
df.drop_duplicates(['ClaimID', 'ComponentSet'],keep= 'last', inplace = True)


In [7]:
df.columns

Index(['ClaimID', 'AGMID', 'LOB', 'State', 'ProgramCode', 'PlanCode',
       'VehicleMake', 'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ComponentCode', 'ComponentSet', 'ContractStatus', 'AgeAtClaim',
       'AgeAtCancel', 'Claimstatus', 'ClaimDate', 'PaidDate', 'AgeofPayment'],
      dtype='object')

In [8]:
nulls = df.isnull().sum()
nulls[nulls > 0]

State           1079158
VehicleMake          93
VehicleModel       3097
ComponentSet         65
AgeAtClaim            1
Claimstatus     1706189
ClaimDate           232
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1706189 entries, 0 to 2505571
Data columns (total 20 columns):
ClaimID           int64
AGMID             int64
LOB               object
State             object
ProgramCode       object
PlanCode          object
VehicleMake       object
VehicleModel      object
VehicleYear       int64
TermMonth         int64
TermMiles         int64
ComponentCode     object
ComponentSet      object
ContractStatus    object
AgeAtClaim        float64
AgeAtCancel       int64
Claimstatus       float64
ClaimDate         float64
PaidDate          int64
AgeofPayment      object
dtypes: float64(3), int64(7), object(10)
memory usage: 273.4+ MB


In [11]:
df.shape

(1706189, 20)

In [12]:
from sklearn.utils import shuffle
df = shuffle(df)

In [13]:
df.columns,df.shape

(Index(['ClaimID', 'AGMID', 'LOB', 'State', 'ProgramCode', 'PlanCode',
        'VehicleMake', 'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
        'ComponentCode', 'ComponentSet', 'ContractStatus', 'AgeAtClaim',
        'AgeAtCancel', 'Claimstatus', 'ClaimDate', 'PaidDate', 'AgeofPayment'],
       dtype='object'), (1706189, 20))

# Missing values at individual variable level

In [16]:
#shows the missing values at individual variable level

#df.isnull().sum() --> works same as below code
df.apply(lambda x: sum(x.isnull()), axis=0)

ClaimID                 0
AGMID                   0
LOB                     0
State             1079158
ProgramCode             0
PlanCode                0
VehicleMake            93
VehicleModel         3097
VehicleYear             0
TermMonth               0
TermMiles               0
ComponentCode           0
ComponentSet           65
ContractStatus          0
AgeAtClaim              1
AgeAtCancel             0
Claimstatus       1706189
ClaimDate             232
PaidDate                0
AgeofPayment            0
dtype: int64

In [17]:
#shows the total number of missing values
df.isnull().sum().sum()

2788835

In [18]:
#variables which we can remove
var_null_pc = df.isnull().sum(axis=0).sort_values( ascending=False)/float(len(df) )
var_null_pc[ var_null_pc > 0.75 ]

Claimstatus    1.0
dtype: float64

In [1]:
df.drop( var_null_pc[ var_null_pc > 0.75 ].index, axis = 1,  ) 
df.dropna( axis = 0, thresh = 30, )

NameError: name 'df' is not defined

# COLUMNS REMOVED

In [20]:
#The Variables which can be removed are -,
vars_to_be_removed = [ 'ClaimDate', 'PaidDate', 'AGMID',  'State','AgeofPayment','AgeAtCancel',
 'AgeAtClaim','ComponentCode'
 ]

df.drop(vars_to_be_removed , axis = 1, inplace=True )#inplace=True

In [21]:
#shows the variable names present in the dataframe
list(df.columns.values)

['ClaimID',
 'LOB',
 'ProgramCode',
 'PlanCode',
 'VehicleMake',
 'VehicleModel',
 'VehicleYear',
 'TermMonth',
 'TermMiles',
 'ComponentSet',
 'ContractStatus',
 'Claimstatus']

In [22]:
df.columns

Index(['ClaimID', 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles', 'ComponentSet',
       'ContractStatus', 'Claimstatus'],
      dtype='object')

In [23]:
# find categorical variables
categorical = [var for var in df.columns if df[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))

There are 7 categorical variables


In [24]:

# find numerical variables
numerical = [var for var in df.columns if df[var].dtype!='O']
print('There are {} numerical variables'.format(len(numerical)))

There are 5 numerical variables


In [25]:
df[categorical].columns

Index(['LOB', 'ProgramCode', 'PlanCode', 'VehicleMake', 'VehicleModel',
       'ComponentSet', 'ContractStatus'],
      dtype='object')

In [26]:
df[numerical].columns

Index(['ClaimID', 'VehicleYear', 'TermMonth', 'TermMiles', 'Claimstatus'], dtype='object')

In [27]:
df.shape

(1706189, 12)

# One hot encoding and dummy encoding on Categorical

In [28]:
#Changing the datatype of the variable in a dataframe
df['ProgramCode'] = df['ProgramCode'].astype('category')
#df['ProductCode'] = df['ProductCode'].astype('category')
df['PlanCode'] = df['PlanCode'].astype('category')

#df['State'] = df['State'].astype('category')
df['ContractStatus'] = df['ContractStatus'].astype('category')
df['ComponentSet'] = df['ComponentSet'].astype('category')
df['LOB'] = df['LOB'].astype('category')
df['VehicleMake'] = df['VehicleMake'].astype('category')
df['VehicleModel'] = df['VehicleModel'].astype('category')
#df['FuelType'] = df['FuelType'].astype('category') 
#df['ComponentCode'] = df['ComponentCode'].astype('category')

In [29]:
# Convert all non-numeric values to number
cat=['LOB',  'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'ContractStatus']

In [30]:
from sklearn import preprocessing
le =preprocessing.LabelEncoder()
for var in cat:
    le = preprocessing.LabelEncoder()
    df[var]=le.fit_transform(df[var].astype('str'))
df.dtypes

ClaimID              int64
LOB                  int32
ProgramCode          int32
PlanCode             int32
VehicleMake          int32
VehicleModel         int32
VehicleYear          int64
TermMonth            int64
TermMiles            int64
ComponentSet      category
ContractStatus       int32
Claimstatus        float64
dtype: object

In [31]:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder

In [32]:
df.isnull().sum()

ClaimID                 0
LOB                     0
ProgramCode             0
PlanCode                0
VehicleMake             0
VehicleModel            0
VehicleYear             0
TermMonth               0
TermMiles               0
ComponentSet           65
ContractStatus          0
Claimstatus       1706189
dtype: int64

In [2]:

# outlies in discrete variables
for var in [  'VehicleYear', 'TermMonth', 'TermMiles',    
       ]:
    print(df[var].value_counts() / np.float(len(df)))
    print()

NameError: name 'df' is not defined

In [34]:
for var in categorical:
    print(var, ' contains ', len(df[var].unique()), ' labels')

LOB  contains  3  labels
ProgramCode  contains  282  labels
PlanCode  contains  738  labels
VehicleMake  contains  139  labels
VehicleModel  contains  11259  labels
ComponentSet  contains  76  labels
ContractStatus  contains  10  labels


In [3]:
df2=pd.concat([df, pd.get_dummies(df.ComponentSet)], 1).groupby(['ClaimID']).sum().reset_index()
df2

NameError: name 'pd' is not defined

In [36]:
df2.drop([ 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ContractStatus', 'Claimstatus'], axis=1, inplace=True) 

In [4]:
df1=df.groupby(['ClaimID']).last()
df1

NameError: name 'df' is not defined

In [38]:
df1.drop([ 'ComponentSet'], axis=1, inplace=True) 


In [39]:
df1.columns

Index(['LOB', 'ProgramCode', 'PlanCode', 'VehicleMake', 'VehicleModel',
       'VehicleYear', 'TermMonth', 'TermMiles', 'ContractStatus',
       'Claimstatus'],
      dtype='object')

In [5]:
df3=df1.merge(df2, left_on='ClaimID', right_on='ClaimID')
df3.columns

NameError: name 'df1' is not defined

In [44]:
nulls = df.isnull().sum()
nulls[nulls > 0]

ComponentSet         65
Claimstatus     1706189
dtype: int64

In [42]:
df3.shape

(1060685, 86)

In [43]:
df3.to_csv('C:/Users/Pavan/Documents/New_train_datav3.csv',index=False)

# END TRAIN DATA CleanUP